# Documentación de Herramientas del Agente de Devoluciones

## 📋 Descripción General

El agente de devoluciones utiliza 4 herramientas principales para interactuar con sistemas externos y procesar solicitudes de devolución. Cada herramienta está diseñada para una función específica dentro del flujo de trabajo.



## 🤖 Arquitectura de LLMs del Agente

### 1. LLM Principal (get_llm())


In [ ]:
# Configuración
model = "meta-llama/llama-3.3-70b-instruct:free"
base_url = "https://openrouter.ai/api/v1"
timeout = 60
max_retries = 3

Propósito: Procesamiento principal de consultas complejas y razonamiento avanzado.

Características:

✅ Modelo grande: 70B parámetros para tareas complejas

✅ Alta capacidad: Razonamiento detallado y comprensión contextual

✅ Tolerante a latencia: Timeout de 60 segundos

✅ Robusto: 3 reintentos automáticos

Uso en el Sistema:

- Consultas RAG complejas

- Toma de decisiones de elegibilidad

- Análisis de políticas detalladas

### 2. LLM Pequeño (get_llm_small())

In [ ]:
# Configuración  
model = "qwen/qwen-2.5-7b-instruct:free"
timeout = 45
max_retries = 2

Propósito: Tareas livianas que no requieren gran capacidad de razonamiento.

Características:

⚡ Modelo eficiente: 7B parámetros para baja latencia

⚡ Respuesta rápida: Timeout de 45 segundos

⚡ Costo optimizado: Modelo gratuito/open-source

⚡ Tareas específicas: Extracción, redacción, formateo

Casos de Uso:

- Extracción de order_id desde texto libre

- Redacción de respuestas amigables al usuario

- Generación de mensajes de cortesía

- Formateo de respuestas estructuradas

### 3. LLM del Sistema RAG (build_policy_rag())

In [ ]:
# Configuración RAG
embeddings = "BAAI/bge-m3"
chunking = "SemanticChunker"
vectorstore = "FAISS"
k = 4  # documentos recuperados

Propósito: Consulta especializada en políticas de devolución usando Retrieval-Augmented Generation.

Características:

🧠 Embeddings avanzados: Modelo BGE-M3 para representación semántica

📚 Chunking inteligente: Segmentación semántica (percentil 65)

🔍 Búsqueda vectorial: FAISS para recuperación eficiente

📖 Contexto relevante: 4 documentos más relevantes por consulta

Flujo de Trabajo RAG:

1. Embedding: Texto → vectores con BGE-M3

2. Chunking: División semántica de políticas

3. Retrieval: Búsqueda por similitud en FAISS

4. Generation: Respuesta contextual con LLM principal

## 🔄 Distribución de LLMs por Herramienta

| Herramienta | LLM Utilizado | Propósito |
|-------------|---------------|-----------|
| `buscar_pedido` | ❌ Ninguno | Consulta directa a DataFrame |
| `verificar_elegibilidad_producto` | ✅ **LLM Principal + RAG** | Análisis semántico de políticas |
| `generar_etiqueta_devolucion` | ❌ Ninguno | Función determinística |
| `registrar_evento_sql` | ❌ Ninguno | Operación de base de datos |
| `extract_order_id_llm` | ✅ **LLM Pequeño** | Extracción rápida de IDs |
| `friendly_ask_for_order_id` | ✅ **LLM Pequeño** | Redacción de mensajes |
| `render_final_reply_with_llm` | ✅ **LLM Pequeño** | Respuestas amigables al usuario |

## 🛠️ 1. Tool: `buscar_pedido`

### **Propósito**
Busca y recupera información básica de un pedido en la base de datos de EcoMarket usando el order_id proporcionado.

### **Esquema de Entrada**
```python
class BuscarPedidoInput(BaseModel):
    order_id: str = Field(..., description="ID del pedido")
```

### **Parámetros de Entrada**
| Parámetro | Tipo | Requerido | Descripción |
|-----------|------|-----------|-------------|
| `order_id` | string | ✅ | Identificador único del pedido (ej: "2509006", "ORD12345") |

### **Respuesta de Salida**
```python
{
    "order_id": "2509006",
    "name": "Juan Pérez",
    "status": "Entregado", 
    "category": "Electrónicos",
    "raw": { ... }  # Datos completos del DataFrame
}
```

### **Comportamiento**
- ✅ **Encontrado**: Devuelve diccionario con información estructurada del pedido
- ❌ **No encontrado**: Devuelve diccionario vacío `{}`
- 🔄 **Error**: Maneja excepciones y retorna `{}` en caso de fallo

## 🛠️ 2. Tool: `verificar_elegibilidad_producto`

### **Propósito**
Consulta el sistema RAG de políticas para determinar si un producto es elegible para devolución basándose en su estado y categoría.

### **Esquema de Entrada**
```python
class VerificarElegibilidadInput(BaseModel):
    order_id: str
    status: str  
    category: str
```

### **Parámetros de Entrada**
| Parámetro | Tipo | Requerido | Descripción |
|-----------|------|-----------|-------------|
| `order_id` | string | ✅ | ID del pedido para trazabilidad |
| `status` | string | ✅ | Estado actual del producto (ej: "Entregado", "En tránsito") |
| `category` | string | ✅ | Categoría del producto (ej: "Electrónicos", "Ropa") |

### **Respuesta de Salida**
```python
{
    "eligible": True,
    "reason": "El producto cumple con todos los criterios de devolución..."
}
```

### **Comportamiento**
- 🤖 **Consulta RAG**: Usa embeddings semánticos con modelo BGE-M3
- 📋 **Basado en políticas**: Decide según políticas de devolución
- 🎯 **Estructurado**: Siempre devuelve `{"eligible": bool, "reason": str}`
- ⚠️ **Fallback**: Si hay error, retorna `eligible: False` con mensaje descriptivo

## 🛠️ 3. Tool: `generar_etiqueta_devolucion`

### **Propósito**
Genera un RMA (Return Merchandise Authorization) y una etiqueta de devolución simulada para productos elegibles.

### **Esquema de Entrada**
```python
class GenerarEtiquetaInput(BaseModel):
    order_id: str
    customer_name: Optional[str] = ""
    category: Optional[str] = ""  
    status: Optional[str] = ""
```

### **Parámetros de Entrada**
| Parámetro | Tipo | Requerido | Descripción |
|-----------|------|-----------|-------------|
| `order_id` | string | ✅ | ID del pedido para generar RMA único |
| `customer_name` | string | ❌ | Nombre del cliente (opcional) |
| `category` | string | ❌ | Categoría del producto (opcional) |
| `status` | string | ❌ | Estado del producto (opcional) |

### **Respuesta de Salida**
```python
{
    "rma": "RMA-20241201-ABC12345",
    "label_url": "https://labels.ecomarket.test/RMA-20241201-ABC12345.pdf",
    "label_text": "=== ETIQUETA DE DEVOLUCIÓN ===\\nRMA: RMA-20241201-ABC12345\\n..."
}
```

## 🛠️ 4. Tool: `registrar_evento_sql`

### **Propósito**
Registra eventos de devolución en base de datos SQLite para auditoría y seguimiento.

### **Esquema de Entrada**
```python
class RegistrarEventoSQLInput(BaseModel):
    order_id: str
    rma: str
    status: Literal["Procesando", "Entregado", ...]  # 8 estados posibles
    notes: Optional[str] = None
    reason: Optional[str] = None  
    category: Optional[str] = None
    customer_name: Optional[str] = None
```

### **Estados Permitidos**
- `"Procesando"`, `"En preparación"`, `"En tránsito"`, `"Retrasado"`
- `"Entregado"`, `"Intento de entrega fallido"`, `"Retenido por aduana"`, `"En revisión de pago"`

### **Respuesta de Salida**
```python
{
    "ok": True,
    "event_id": "uuid-unico",
    "ts": "2024-12-01T10:30:00",
    "order_id": "2509006",
    ...  # todos los campos del evento
}
```

## 🔄 Resumen de Integración

### **Secuencia Típica del Flujo**
1. `buscar_pedido` → Recupera datos del pedido
2. `verificar_elegibilidad_producto` → Consulta políticas via RAG  
3. `generar_etiqueta_devolucion` → Crea RMA y etiqueta (si es elegible)
4. `registrar_evento_sql` → Registra evento en BD para trazabilidad

### **Características Comunes de Todas las Tools**
- ✅ **Tipado fuerte**: Usan Pydantic para validación de esquemas
- 🔧 **Decoradas con `@tool`**: Integración nativa con LangGraph
- 📝 **Documentación completa**: Docstrings detallados
- 🛡️ **Robustas**: Manejo de errores y fallbacks integrados
- 🔄 **Estructuras consistentes**: Siempre retornan diccionarios predecibles

### **Manejo de Errores**
- Todas las tools incluyen manejo robusto de excepciones
- Retornan estructuras consistentes incluso en caso de fallo
- No interrumpen el flujo principal del agente
- Proporcionan mensajes de error descriptivos

## 🎯 Ejemplo de Flujo Completo

### **Caso: Devolución Exitosa**
```python
# 1. Buscar pedido
pedido = buscar_pedido.invoke({"order_id": "2509006"})
# Returns: {"order_id": "2509006", "status": "Entregado", "category": "Electrónicos"}

# 2. Verificar elegibilidad
elegibilidad = verificar_elegibilidad_producto.invoke({
    "order_id": "2509006", 
    "status": "Entregado", 
    "category": "Electrónicos"
})
# Returns: {"eligible": True, "reason": "Producto cumple políticas..."}

# 3. Generar etiqueta (si es elegible)
if elegibilidad["eligible"]:
    etiqueta = generar_etiqueta_devolucion.invoke({
        "order_id": "2509006",
        "customer_name": pedido["name"]
    })
    # Returns: {"rma": "RMA-20241201-ABC123", "label_url": "..."}

# 4. Registrar evento
registro = registrar_evento_sql.invoke({
    "order_id": "2509006",
    "rma": etiqueta["rma"],
    "status": "Procesando",
    "notes": "Devolución aprobada"
})
# Returns: {"ok": True, "event_id": "...", ...}
```

### **Notas de Implementación**
- Las tools son ejercicios en baja escala pero representan integraciones reales
- Pueden ser extendidas para conectar con APIs corporativas
- El RAG usa embeddings BGE-M3 con FAISS para búsqueda semántica
- La base de datos SQLite permite auditoría y reporting

## 🔧 Información Técnica Adicional

### **Dependencias Principales**
- **LangChain/LangGraph**: Framework para agentes y tools
- **Pydantic**: Validación de esquemas y tipos
- **FAISS**: Almacenamiento vectorial para RAG
- **SQLite**: Base de datos para eventos
- **HuggingFace Embeddings**: Modelo BGE-M3 para embeddings

### **Estructura de Archivos**
```
/project_root/
├── data/
│   ├── documents/           # Políticas en MD
│   ├── vectorstore/         # Índice FAISS
│   └── data_log/           # BD SQLite
├── codigo_2.py             # Implementación agentes
└── app_streamlit.py        # Interfaz web
```

### **Flujo grama del proceso**

[![](https://mermaid.ink/img/pako:eNpNUl1v2yAU_SvoPrWS28U2SWM_VIpaaS-pKkXuy0wVUXPjoNrgYci6JfnvBdxle-Oer8sRHKHRAqGEXad_NXtuLKkemXpJr-qX0XEj9es1U5u03mhn0RCBRHCrx1emVmm9Xj-VZEAhTUTJN2JwHLQSaLygSuvq-Xldkjc3Ntxsg1DoQGQ1g4k6oJE7GVjssJVvspOCi-1gtHCNT7xyIyeb1fdrBsGYfyW2qNAEk5U_HVq-FXjQnWukVkFGv2TGR442Cg-obNgdupGbm_uTNv6WWykSMr67hPTayoM-kU0a6kbFjneWx2InsvoH6_cTqdJQL45KE_zwW3ASTSipslDzIojdOi-p6AX_D8xDtckXeBqPIQ0SaI0UUFrjMIEeTc_DCEemCGFg99gjg9IfBe646ywDps7eNnD1Q-v-r9No1-6h9JVGP7nBt8JHyVvD-wtqMDzcg3bKQpnRmAHlET6gpLP8dpnOl0U-o_P0Lvfkb6_JPJjlaVYss2JWzOk5gT9x6ex2mdG8yBZ3i3ye0zldJOBf32rzNH23-OvOn_hCzzU?type=png)](https://mermaid.live/edit#pako:eNpNUl1v2yAU_SvoPrWS28U2SWM_VIpaaS-pKkXuy0wVUXPjoNrgYci6JfnvBdxle-Oer8sRHKHRAqGEXad_NXtuLKkemXpJr-qX0XEj9es1U5u03mhn0RCBRHCrx1emVmm9Xj-VZEAhTUTJN2JwHLQSaLygSuvq-Xldkjc3Ntxsg1DoQGQ1g4k6oJE7GVjssJVvspOCi-1gtHCNT7xyIyeb1fdrBsGYfyW2qNAEk5U_HVq-FXjQnWukVkFGv2TGR442Cg-obNgdupGbm_uTNv6WWykSMr67hPTayoM-kU0a6kbFjneWx2InsvoH6_cTqdJQL45KE_zwW3ASTSipslDzIojdOi-p6AX_D8xDtckXeBqPIQ0SaI0UUFrjMIEeTc_DCEemCGFg99gjg9IfBe646ywDps7eNnD1Q-v-r9No1-6h9JVGP7nBt8JHyVvD-wtqMDzcg3bKQpnRmAHlET6gpLP8dpnOl0U-o_P0Lvfkb6_JPJjlaVYss2JWzOk5gT9x6ex2mdG8yBZ3i3ye0zldJOBf32rzNH23-OvOn_hCzzU)

# Selección del Marco de Agentes: LangChain + LangGraph

## 🎯 Decisión Técnica
**Marco Seleccionado**: LangChain + LangGraph  
**Alternativa Considerada**: LlamaIndex  
**Justificación**: Optimización para flujos de trabajo complejos con múltiples herramientas y estado persistente.

---

## 📊 Comparativa de Marcos

| Característica | LangChain + LangGraph | LlamaIndex |
|----------------|----------------------|------------|
| **Flujos de trabajo complejos** | ✅ **Excelente** (StateGraph) | ⚠️ Limitado |
| **Integración de herramientas** | ✅ **Nativa** (@tool decorator) | ✅ Buena |
| **Manejo de estado** | ✅ **Robusto** (AgentState) | ❌ Básico |
| **Grafos ejecutables** | ✅ **Especializado** (StateGraph) | ❌ No aplica |
| **Checkpoints y memoria** | ✅ **Avanzado** (MemorySaver) | ⚠️ Limitado |
| **RAG integrado** | ✅ **Completo** | ✅ **Excelente** |
| **Curva de aprendizaje** | ⚠️ Moderada | ✅ Suave |
| **Documentación** | ✅ **Extensa** | ✅ Buena |

---

## 🚀 Justificación para el MVP Actual

### **1. Flujo de Trabajo Complejo y Secuencial**
```python
FLUJO_AGENTE = [
    "router_node" → "buscar_pedido_node" → "verificar_elegibilidad_producto" 
    → "generar_etiqueta_devolucion" → "registrar_evento_sql" → "responder_node"
]

**LangGraph permite:**

✅ Grafos condicionales: Rutas dinámicas basadas en estado

✅ Estado persistente: Mantener contexto entre nodos

✅ Checkpoints: Recuperación de conversaciones

**Ventajas:**

✅ Decoradores nativos: @tool para definición rápida

✅ Esquemas Pydantic: Validación automática de entradas

✅ Registro automático: Las tools se descubren automáticamente

**Capacidades críticas:**

✅ Estado tipado: Type hints para desarrollo seguro

✅ Persistencia: MemorySaver para conversaciones largas

✅ Serialización: Fácil debug y logging

🎯 Por Qué No LlamaIndex para Este MVP
Limitaciones de LlamaIndex:

❌ Flujos de Trabajo Complejos:

- Enfoque en RAG, no en orquestación de agentes

- MVP requiere 7 nodos interconectados con lógica condicional

❌ Manejo de Estado Avanzado:

- Necesitamos mantener 10+ campos de estado

- LangGraph provee AgentState out-of-the-box

❌ Grafos Ejecutables:

- "Devolución de productos" requiere flujo secuencial estricto

- LangGraph permite flujos declarativos

💡 Casos Donde LlamaIndex Sería Mejor
Consideraríamos LlamaIndex si:

🔍 Solo búsqueda semántica: Sin flujos complejos

📚 RAG puro: Consultas documentales sin estado

🚀 Prototipo rápido: Menos de 3 herramientas simples

🔗 Data connectors: Fuentes de datos heterogéneas

## 🏆 Conclusión: LangChain + LangGraph es la Elección Óptima

### **Para Nuestro MVP de Devoluciones**:

| Requerimiento | LangGraph | LlamaIndex |
|---------------|-----------|------------|
| **7 herramientas integradas** | ✅ **Perfecto** | ⚠️ Posible |
| **Flujo condicional complejo** | ✅ **Ideal** | ❌ Limitado |
| **Estado persistente** | ✅ **Nativo** | ❌ Básico |
| **RAG para políticas** | ✅ **Suficiente** | ✅ **Excelente** |
| **Checkpoints conversación** | ✅ **Incluido** | ❌ No aplica |
| **Time to Market** | ✅ **Rápido** | ⚠️ Moderado |

### Decisión Final:
✅ LangChain + LangGraph es la elección técnica correcta porque nuestro MVP es fundamentalmente un sistema de flujo de trabajo con estado que incidentalmente usa RAG, no un sistema de RAG que incidentalmente tiene flujo de trabajo.

Ventajas clave:

🎯 Flujo visual claro: Grafos fáciles de entender y modificar

🔧 Mantenibilidad: Estado tipado y herramientas organizadas

📈 Escalabilidad: Fácil añadir nuevas herramientas y nodos

🐛 Debugging: Mejor trazabilidad y logs estructurados



# Reflexión Crítica sobre el Agente de Devoluciones

## 🔒 Análisis de Seguridad y Ética

### **Riesgos Identificados**
La implementación de un agente de IA con capacidad de tomar acciones autónomas introduce nuevos riesgos éticos y de seguridad que deben ser mitigados proactivamente. Nuestro agente de devoluciones, aunque opera en un contexto limitado, presenta varios escenarios de riesgo potencial:

**Riesgo 1: Toma de Decisiones Incorrectas sobre Elegibilidad**
- *Escenario*: El RAG podría malinterpretar políticas y aprobar devoluciones no permitidas o rechazar casos legítimos
- *Impacto*: Pérdidas económicas para la empresa o insatisfacción del cliente
- *Mitigación*: Implementar umbrales de confianza y revisión humana para casos borderline

**Riesgo 2: Exposición de Datos Sensibles**
- *Escenario*: El agente podría accidentalmente revelar información de otros clientes o datos internos sensibles
- *Impacto*: Violación de privacidad y posibles sanciones legales
- *Mitigación*: Filtrado estricto de respuestas y máscara de datos personales en logs

**Riesgo 3: Dependencia Excesiva del Cliente**
- *Escenario*: Los usuarios podrían confiar ciegamente en las decisiones del agente sin verificación
- *Impacto*: Errores propagados y pérdida de interacción humana crítica
- *Mitigación*: Comunicación clara de limitaciones y opción de escalar a agente humano

### **Principios Éticos Implementados**
- **Transparencia**: El agente explica sus decisiones basándose en políticas específicas
- **Auditabilidad**: Todas las interacciones se registran en SQLite para posterior revisión
- **Proporcionalidad**: Las acciones están limitadas al ámbito de devoluciones
- **Control Humano**: Siempre existe la opción de contactar servicio al cliente humano

## 📊 Monitoreo y Observabilidad

### **Sistema de Registro Implementado**
Hemos diseñado un sistema multicapa de monitoreo que garantiza la trazabilidad completa del agente:

**Capa 1: Registro de Eventos en SQLite**

In [ ]:
-- Tabla eventos_devoluciones.db
-- Registra: timestamp, order_id, acción realizada, resultado, razones de decisión
-- Permite: Auditoría posterior, análisis de tendencias, detección de anomalías

 **Capa 2: Logs Estructurados de Ejecución**

- Cada tool genera logs con entrada, salida y métricas de performance

- Seguimiento del flujo completo a través del StateGraph

- Métricas de latencia por herramienta y por LLM

**Capa 3: Sistema de Alertas Propuesto**

In [ ]:
SISTEMA_ALERTAS = {
    "umbral_rechazos": ">80% de devoluciones rechazadas en 1 hora",
    "latencia_pico": "Tiempo de respuesta > 30 segundos consistentemente", 
    "errores_consecutivos": ">5 errores sequentiales en misma herramienta",
    "anomalia_decisiones": "Patrón inusual en criterios de elegibilidad"
}

**Capa 4: Dashboard de Monitoreo**

- Tasa de éxito por herramienta

- Tiempos de respuesta promedio

- Distribución de decisiones (aprobado/rechazado)

- Uso de recursos por modelo de LLM

### 🚀 Propuestas de Mejora y Evolución

**Agente de Reemplazo Automático**

Problema actual: El proceso de reemplazo requiere intervención manual después de la devolución
Solución propuesta:

In [ ]:
@tool("generar_orden_reemplazo", args_schema=ReemplazoInput)
def generar_orden_reemplazo(order_id: str, producto_original: str, motivo: str):
    # Consultar inventario en tiempo real
    # Verificar disponibilidad de producto similar
    # Generar orden de reemplazo automática
    # Notificar al cliente por email

Beneficios:

- Reducción de 3-5 días en el proceso de reemplazo

- Mejora en la experiencia del cliente

- Disminución de carga en servicio al cliente

**Agente de Actualización de CRM**

Problema actual: La información del cliente se actualiza manualmente después de interacciones
Solución propuesta:

In [ ]:
@tool("actualizar_perfil_cliente", args_schema=CRMInput) 
def actualizar_perfil_cliente(cliente_id: str, interaccion: str, resultado: str, satisfaccion: int):
    # Extraer insights de la conversación
    # Actualizar preferencias del cliente
    # Marcar tendencias de comportamiento
    # Sugerir acciones de retención si es necesario

Beneficios:

- CRM siempre actualizado con interacciones recientes

- Mejor personalización en futuras comunicaciones

- Detección temprana de clientes en riesgo

**Sistema de Escalación Inteligente**

Problema actual: No hay criterios claros para escalar a agente humano
Solución propuesta:

In [ ]:
def deber_escalar_humano(estado_agente: AgentState) -> bool:
    criterios = [
        "confianza_decicion < 0.7",
        "intentos_fallidos > 2", 
        "cliente_expresa_frustracion == True",
        "caso_complejidad_alta == True"
    ]
    return any(criterios)

**Análisis Predictivo de Devoluciones**

Propuesta avanzada:

In [ ]:
@tool("predecir_riesgo_devolucion", args_schema=PrediccionInput)
def predecir_riesgo_devolucion(cliente_id: str, producto: str, historial: Dict):
    # Analizar patrones históricos
    # Identificar productos con alta tasa de devolución
    # Sugerir acciones preventivas
    # Alertar sobre posibles abusos del sistema

### 💡 Conclusión Reflexiva
La implementación de agentes de IA autónomos representa un equilibrio delicado entre eficiencia operativa y responsabilidad ética. Nuestro agente de devoluciones demuestra que es posible crear sistemas útiles manteniendo safeguards apropiados.

Lecciones aprendidas:

1. La transparencia en la toma de decisiones es tan importante como la precisión

2. Los sistemas de monitoreo deben diseñarse desde el día uno, no como afterthought

3. La escalación humana debe ser una característica fundamental, no un plan de contingencia

**Visión futura:** Creemos que los agentes especializados como este representan el futuro de la atención al cliente - no reemplazando humanos, sino amplificando sus capacidades y permitiéndoles enfocarse en casos que realmente requieren empatía y juicio complejo.

El camino forward incluye expandir las capacidades del agente mientras fortalecemos simultáneamente los frameworks éticos y de seguridad que garantizan su operación responsable.